In [1]:
## Please add your Open AI licence Key here
import os
os.environ["OPENAI_API_KEY"] ="sk-XXX"

In [ ]:
## Using https://www.trychroma.com/
## Using langchain
## Plenty of integrations https://python.langchain.com/docs/modules/data_connection/document_loaders/integrations/arxiv
## Using openai gpt-3.5 model
## Idea respectfully taken from https://github.com/devsentient/examples/blob/main/LLMs/confluence_app/confluenceQA.ipynb

In [2]:
# Constants
EMB_OPENAI_ADA = "text-embedding-ada-002"
LLM_OPENAI_GPT35 = "gpt-3.5-turbo"

In [3]:
## Installations 
!pip install langchain==0.0.189
!pip install chromadb==0.3.25
!pip install openai==0.27.6
!pip install pytesseract==0.3.10
!pip install beautifulsoup4==4.12.2
!pip install atlassian-python-api==3.38.0
!pip install tiktoken==0.4.0
!pip install lxml==4.9.2

  Using cached chromadb-0.3.25-py3-none-any.whl (86 kB)
  Using cached hnswlib-0.7.0-cp310-cp310-macosx_11_0_arm64.whl
  Using cached clickhouse_connect-0.6.4-cp310-cp310-macosx_11_0_arm64.whl (231 kB)
  Using cached duckdb-0.8.1-cp310-cp310-macosx_11_0_arm64.whl (12.6 MB)
  Using cached fastapi-0.98.0-py3-none-any.whl (56 kB)
  Using cached posthog-3.0.1-py2.py3-none-any.whl (37 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-macosx_12_0_arm64.whl (3.9 MB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached typing_extensions-4.7.0-py3-none-any.whl (33 kB)
  Using cached overrides-7.3.1-py3-none-any.whl (17 kB)
  Using cached zstandard-0.21.0-cp310-cp310-macosx_11_0_arm64.whl (364 kB)
  Using cached starlette-0.27.0-py3-none-any.whl (66 kB)


  Using cached watchfiles-0.19.0-cp37-abi3-macosx_11_0_arm64.whl (388 kB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.3
    Uninstalling tqdm-4.62.3:
      Successfully uninstalled tqdm-4.62.3
  Attempting uninstall: starlette
    Found existing installation: starlette 0.19.1
    Uninstalling starlette-0.19.1:
      Successfully uninstalled starlette-0.19.1
  Attempting uninstall: fastapi
    Found existing installation: fastapi 0.84.0
    Uninstalling fastapi-0.84.0:
      Successfully uninstalled fastapi-0.84.0
ERROR: pip's dependency resolver does not currently take into account all the packages

In [4]:
import os
from langchain.document_loaders import ConfluenceLoader
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

In [5]:
embedding = OpenAIEmbeddings()

In [10]:
llm = ChatOpenAI(model_name=LLM_OPENAI_GPT35, temperature=0.)

In [14]:
## Used to extract plain text from the html page https://quepid.com/docs/
from bs4 import BeautifulSoup
import requests

def extract_text_from(url):
    html = requests.get(url).text
    soup = BeautifulSoup(html, features="html.parser")
    text = soup.get_text()

    lines = (line.strip() for line in text.splitlines())
    return '\n'.join(line for line in lines if line)

In [21]:
## will go in config file in app

config = {"persist_directory":"./chroma_db/",
          "doc_url":"https://quepid.com/docs/"
          }

In [22]:
## Extract plain text from html
doc_text = extract_text_from(config.get("doc_url",None))
print(doc_text)

Documentation | Quepid
Toggle navigation
Docs
Tutorials
Documentation
Return to Quepid
Back to top
Getting Started
Welcome to Quepid, your search relevancy toolkit. Getting started is easy and we will walk you through the concepts and features to get you setup and tuning relevance in no time.
Core Concepts
First, let's walk through a few of the core concepts in Quepid.
Case:
A case refers to all of the queries and relevance tuning settings for a single search engine. If you want to work with multiple instances of Solr or Elasticsearch, you must create a separate case for each one.
Query:
Within a case, queries are the keywords or other search criteria and their corresponding set of results that will be rated to determine the overall score of a case.
Rating:
Ratings are the numerical values given to a result that indicates how relevant a particular search result is for the query. How each rating is interpreted depends on the scorer used for the query (or case), but usually the higher th

In [27]:
## We use langchain here to split text into chunks of 225 tokens each

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=225, separator="\n")
#docs = text_splitter.split_documents(doc_text)
splits = text_splitter.split_text(doc_text)
print(f"Splits {splits} into {len(splits)} chunks")
##Print splitted chunks and number of chunks "into 135 chunks" in our case.

Created a chunk of size 273, which is longer than the specified 225
Created a chunk of size 270, which is longer than the specified 225
Created a chunk of size 296, which is longer than the specified 225
Created a chunk of size 286, which is longer than the specified 225
Created a chunk of size 278, which is longer than the specified 225
Created a chunk of size 288, which is longer than the specified 225
Created a chunk of size 229, which is longer than the specified 225
Created a chunk of size 248, which is longer than the specified 225
Created a chunk of size 304, which is longer than the specified 225
Created a chunk of size 268, which is longer than the specified 225
Created a chunk of size 231, which is longer than the specified 225
Created a chunk of size 437, which is longer than the specified 225
Created a chunk of size 471, which is longer than the specified 225
Created a chunk of size 256, which is longer than the specified 225
Created a chunk of size 252, which is longer tha

Splits ['Documentation | Quepid\nToggle navigation\nDocs\nTutorials\nDocumentation\nReturn to Quepid\nBack to top\nGetting Started', 'Return to Quepid\nBack to top\nGetting Started\nWelcome to Quepid, your search relevancy toolkit. Getting started is easy and we will walk you through the concepts and features to get you setup and tuning relevance in no time.', 'Getting Started\nWelcome to Quepid, your search relevancy toolkit. Getting started is easy and we will walk you through the concepts and features to get you setup and tuning relevance in no time.\nCore Concepts', "Core Concepts\nFirst, let's walk through a few of the core concepts in Quepid.\nCase:", 'Case:\nA case refers to all of the queries and relevance tuning settings for a single search engine. If you want to work with multiple instances of Solr or Elasticsearch, you must create a separate case for each one.\nQuery:', 'Query:\nWithin a case, queries are the keywords or other search criteria and their corresponding set of r

In [29]:
## Saving text + embedding in Chroma DB 
persist_directory = config.get("persist_directory",None)
if persist_directory and os.path.exists(persist_directory):
    vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
else:
    vectordb = Chroma.from_texts(texts=splits, embedding=embedding, persist_directory=persist_directory)
    

In [30]:
## Retrieving data from using langchain RetrievalQA
retriever = vectordb.as_retriever(search_kwargs={"k":4})
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff",retriever=retriever)

In [31]:
question = "How to get ratings?"

answer = qa.run(question)
print(answer)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


To get ratings for search results, you can follow these steps:

1. Open the search results page in the search engine or platform you are using.
2. Look for a dropdown or menu option next to each search result. This dropdown usually contains a rating scale.
3. Click on the dropdown to bring up the rating scale.
4. Select the appropriate rating for each search result based on its relevance to your query. The rating scale is typically a numerical scale, such as 1 to 10, where a higher number indicates a more relevant result.
5. Repeat this process for each search result on the page.
6. Some platforms also provide an option to "score all" results the same for a particular query. This means you can assign the same rating to all results with a single click.
7. As you rate each search result, the average score for both the query and the case will be automatically updated.

Note that the specific steps may vary depending on the search engine or platform you are using.


In [32]:
question = "can i use quepid with elasticsearch?"

answer = qa.run(question)
print(answer)

Yes, you can use Quepid with Elasticsearch. Quepid supports both Solr and Elasticsearch as search engines. All you need to do is indicate that you are working with Elasticsearch and provide the URL to your Elasticsearch instance.


In [34]:
question = "can i use quepid with algolia?"

answer = qa.run(question)
print(answer)

No, currently Quepid only supports Solr and Elasticsearch as search engines. Algolia is not supported.


In [35]:
question = "can i have snapshots in quepid?"

answer = qa.run(question)
print(answer)

Yes, you can have snapshots in Quepid. Snapshots allow you to save the current ratings for all of your queries at a specific point in time. This allows you to compare snapshots over time and track the improvement in relevancy. To create a snapshot, click on "Create Snapshot" in the case actions area, give your snapshot a descriptive name, and click "Take Snapshot."


In [36]:
question = "How can quepid help me tuning relevance of my search engine"

answer = qa.run(question)
print(answer)

Quepid can help you tune the relevance of your search engine by providing a toolkit specifically designed for this purpose. It allows you to add queries and view the results, and then modify the search relevancy settings to improve the accuracy and effectiveness of your search results. The relevancy tuning panel in Quepid allows you to make adjustments and fine-tune the relevancy of your search engine.
